In [ ]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("gpt-4o-mini", model_provider="openai")

In [ ]:
from langchain_core.messages import HumanMessage

print(model.invoke([HumanMessage(content="Hi! I'm Bob")]))

# 模型本身并没有状态的概念,不记得你的名字
print(model.invoke([HumanMessage(content="What's my name?")]))

In [ ]:
from langchain_core.messages import AIMessage

print(model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
))

### Message persistence 消息持久化

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [ ]:
config = {"configurable": {"thread_id": "abc123"}}

In [ ]:
query = "Hi! I'm Bob."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

In [ ]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

In [ ]:
# 更换 thread_id
config = {"configurable": {"thread_id": "abc234"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

In [ ]:
# 使用相同的 thread_id 继续对话
config = {"configurable": {"thread_id": "abc123"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

In [ ]:
# 若要支持异步操作，请将 call_model 节点更新为异步函数，并在调用应用时使用 .ainvoke：

# Async function for node:
async def call_model(state: MessagesState):
    response = await model.ainvoke(state["messages"])
    return {"messages": response}


# Define graph as before:
workflow = StateGraph(state_schema=MessagesState)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)
app = workflow.compile(checkpointer=MemorySaver())

# Async invocation:
output = await app.ainvoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

### Prompt templates 提示模板

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You talk like a pirate. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [ ]:
workflow = StateGraph(state_schema=MessagesState)


def call_model(state: MessagesState):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": response}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [ ]:
config = {"configurable": {"thread_id": "abc345"}}
query = "Hi! I'm Jim."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

In [ ]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

In [ ]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [ ]:
# 导入类型注解相关模块
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict


# 定义自定义状态类，扩展 MessagesState
# 添加 language 字段来支持多语言功能
class State(TypedDict):
    # messages 字段使用 add_messages 注解，支持消息列表的自动合并
    messages: Annotated[Sequence[BaseMessage], add_messages]
    # language 字段用于指定回复语言
    language: str


# 使用自定义状态创建工作流
workflow = StateGraph(state_schema=State)


# 更新模型调用函数，支持自定义状态
def call_model(state: State):
    # 使用状态中的所有信息（包括 language）来格式化提示
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    # 返回消息列表（注意这里用的是列表格式）
    return {"messages": [response]}


# 构建工作流图
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# 编译应用
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [ ]:
# 测试多语言功能
config = {"configurable": {"thread_id": "abc456"}}
query = "Hi! I'm Bob."
language = "Spanish"  # 指定回复语言为西班牙语

input_messages = [HumanMessage(query)]
# 传入消息和语言参数
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
# 模型应该用西班牙语回复
output["messages"][-1].pretty_print()

In [ ]:
# 继续对话，测试记忆功能
query = "What is my name?"

input_messages = [HumanMessage(query)]
# 注意：这里没有传入 language 参数
# 但由于使用了相同的 thread_id，之前的语言设置可能会被保留
output = app.invoke(
    {"messages": input_messages},
    config,
)
# 模型应该记住用户名字是 Bob，并可能继续用西班牙语回复
output["messages"][-1].pretty_print()

### Managing Conversation History 管理对话历史

在长时间的对话中，消息历史可能会变得很长，这会增加成本并可能超出模型的上下文限制。
我们可以使用消息修剪功能来管理对话历史的长度。

In [ ]:
# 导入消息修剪相关组件
from langchain_core.messages import SystemMessage, trim_messages

# 创建消息修剪器
trimmer = trim_messages(
    max_tokens=65,        # 最大 token 数量
    strategy="last",      # 保留最后的消息
    token_counter=model,  # 使用模型来计算 token 数量
    include_system=True,  # 包含系统消息
    allow_partial=False,  # 不允许部分消息
    start_on="human",     # 从人类消息开始
)

# 创建一个长对话历史用于测试
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

# 应用修剪器，查看保留的消息
trimmer.invoke(messages)

In [ ]:
# 创建带有消息修剪功能的工作流
workflow = StateGraph(state_schema=State)


# 定义带有调试信息的模型调用函数
def call_model(state: State):
    # 打印修剪前的消息数量
    print(f"Messages before trimming: {len(state['messages'])}")
    # 应用消息修剪
    trimmed_messages = trimmer.invoke(state["messages"])
    # 打印修剪后的消息数量
    print(f"Messages after trimming: {len(trimmed_messages)}")
    print("Remaining messages:")
    # 显示保留的消息内容
    for msg in trimmed_messages:
        print(f"  {type(msg).__name__}: {msg.content}")
    # 使用修剪后的消息和语言信息格式化提示
    prompt = prompt_template.invoke(
        {"messages": trimmed_messages, "language": state["language"]}
    )
    response = model.invoke(prompt)
    return {"messages": [response]}


# 构建工作流图
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# 编译应用
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [ ]:
# 测试消息修剪功能
config = {"configurable": {"thread_id": "abc567"}}
query = "What is my name?"
language = "English"

# 将之前的长对话历史与新消息合并
input_messages = messages + [HumanMessage(query)]
# 调用应用，观察消息修剪的效果
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
# 由于消息被修剪，模型可能不记得用户的名字
output["messages"][-1].pretty_print()

In [ ]:
# 测试另一个问题，验证消息修剪的效果
config = {"configurable": {"thread_id": "abc678"}}

query = "What math problem was asked?"
language = "English"

# 使用相同的长对话历史
input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
# 由于消息修剪，模型应该能够回答关于数学问题的询问（2+2）
# 因为这个问题在修剪后保留的消息中
output["messages"][-1].pretty_print()

### Streaming 流式输出

流式输出允许我们实时接收模型的响应，而不是等待完整的回复。
这对于提供更好的用户体验特别有用，特别是在生成长文本时。

In [ ]:
# 演示流式输出功能
config = {"configurable": {"thread_id": "abc789"}}
query = "Hi I'm Todd, please tell me a joke."
language = "English"

input_messages = [HumanMessage(query)]
# 使用 stream 方法进行流式调用
for chunk, metadata in app.stream(
    {"messages": input_messages, "language": language},
    config,
    stream_mode="messages",  # 设置流模式为消息
):
    # 只处理 AI 消息（过滤掉其他类型的消息）
    if isinstance(chunk, AIMessage):
        # 实时打印每个文本块，用 | 分隔
        print(chunk.content, end="|")